# Checking the Velocity Distribution
## particles coordinates obtained from imageJ

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import trackpy as tp
import pandas as pd
import os

In [3]:
folderPath = r'g:\My Drive\workAppState\prj_shocks\expData\data20230530velDistribTest_ImageJ_Connor'

In [4]:
inputsPath = folderPath + r'\inputs'
infilename = r'\360v_1mA_9sccm_91mtorr'

In [5]:
colnames=['x','y','frame']
indf = pd.read_csv(inputsPath + infilename  + r'.csv', usecols=[3,4,8],names=colnames, skiprows=1)


In [6]:
indf.head()

,x,y,frame
0,345.78586,13.41291,1
1,306.32614,14.01396,1
2,332.94084,13.89708,1
3,319.62251,14.91738,1
4,371.09624,14.78872,1


In [7]:
tracked_csv_dir = folderPath + r'\outputs' + r'\tracked' 
tracked_csv_filepath = tracked_csv_dir + infilename + r'_tracked.csv'
if not os.path.exists(tracked_csv_dir):
    os.makedirs(tracked_csv_dir)
    t_large = tp.link(t_csv,3,memory=1)
    t_large.to_csv(tracked_csv_filepath)    
else:
    t_large = pd.read_csv(tracked_csv_filepath)
t_large


,Unnamed: 0,x,y,frame,particle
0,0,345.78586,13.41291,1,0
1,1299,563.45040,319.49206,1,1
2,1298,513.90593,319.84384,1,2
3,1297,284.83552,319.66066,1,3
4,1296,206.13283,319.97588,1,4
...,...,...,...,...,...
1872186,1870898,210.52854,191.02396,968,1920
1872187,1870897,612.83741,188.58557,968,1541
1872188,1870896,494.81270,189.19780,968,1895
1872189,1870864,114.40066,184.15846,968,1864


In [8]:
#function get_velocities(df, step) calculates velocites of particles
# with a given step step.
def get_velocities(df, step):
#initialize empy arrays to store data:
    arr_particle = np.array([])
    arr_x = np.array([])
    arr_y = np.array([])
    arr_vx = np.array([])
    arr_vy = np.array([])
    arr_frame = np.array([])
    # get an array containing all frame numbers in the input dataframe:
    frames_listing = np.unique(np.array(df['frame']))
    #cycle throught all those frames:
    for iFrame in range(step, len(frames_listing)):
        #get current frame:
        cur_frame = frames_listing[iFrame]
        #select a dataframe containing data ONLY for that frame:
        df_front_frame = df[(df['frame'] == cur_frame)]
        print(iFrame)
        #cycle throught all particles in the frame and find their velocities as
        # position of the particle in that frame minus position of the same particles
        # step frames ago:
        for i in range(0, len(df_front_frame)):
            #take i-th particle in a frame
            cur_p = df_front_frame['particle'].iloc[i]
            cur_x = df_front_frame['x'].iloc[i]
            cur_y = df_front_frame['y'].iloc[i]
            #find a row with the same particle in a frame step frames ago:
            prev_frame_cur_row = df[((df['frame'] == cur_frame - step) & (df['particle'] == cur_p))]
            #if that particle excisted back then, we will get exactly ONE row:
            if (len(prev_frame_cur_row) == 1):
                #if this row exists, we can take position of that particle in that, previous, frame:
                prev_x = prev_frame_cur_row['x'].iloc[0]
                prev_y = prev_frame_cur_row['y'].iloc[0]
                # so we can calculate velocities:
                cur_vx = cur_x - prev_x
                cur_vy = cur_y - prev_y
                cur_particle = df_front_frame['particle'].iloc[i]
                #and append all parameters of that particle to our data arrays
                arr_vx = np.append(arr_vx, cur_vx)
                arr_vy = np.append(arr_vy, cur_vy)
                arr_particle = np.append(arr_particle, cur_particle)
                arr_x = np.append(arr_x, cur_x)
                arr_y = np.append(arr_y, cur_y)
                arr_frame = np.append(arr_frame, cur_frame)
    #save output as a dataframe containing all the info we need:
    data = {'frame':arr_frame, 'particle':arr_particle, 'x': arr_x, 'y': arr_y, 'vx': arr_vx, 'vy':arr_vy}
    ret_df = pd.DataFrame(data)
                
    return ret_df

## Getting the Temperature Profiles

#### constants:

In [9]:
step = 1 # number of frames between the frames for calculating velocities
dust_mass = 4.0 / 3.0 * np.pi * (7.14E-6 / 2.0)**3 * 1510.0 #mass of the dust particles
N_points = 15 # number of points in the temperature profile
rate_for_vel = 99.0 / step # frequency for calculating particle speeds
mm_per_px = 1.0/121.8 # camera resolution
N_bins = 20 #number of bins in the hystograms

#### first, get velocities. May take some time:

In [ ]:
vel_csv_dir = folderPath + r'\outputs\csvData\velocities_step01'
vel_csv_csv_filepath = vel_csv_dir + infilename + r'_velocities_step01.csv'
if not os.path.exists(vel_csv_dir):
    os.makedirs(vel_csv_dir)
    df_vel = get_velocities(t_large, step)
    df_vel.to_csv(vel_csv_csv_filepath)    
else:
    df_vel = pd.read_csv(vel_csv_csv_filepath)
df_vel.head()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


In [ ]:
t_sorted = t_large.sort_values(by = ['particle', 'frame'])

In [ ]:
t_sorted.head()

In [ ]:
grouped = t_sorted.groupby('particle')
grouped.head()

In [ ]:
t_sorted['vx'] = grouped['x'].diff(periods = step)
t_sorted['vy'] = grouped['y'].diff(periods = step)
t_sorted = t_sorted.dropna()
t_sorted.head()

In [ ]:
t_sorted = t_sorted.sort_values(by = ['frame', 'particle'])
t_sorted.head()

In [ ]:
len(t_sorted)

In [ ]:
len(df_vel)

In [ ]:
df_vel = df_vel.sort_values(by = ['frame', 'particle'])
df_vel['particle'] = df_vel['particle'].astype('int')
df_vel['frame'] = df_vel['frame'].astype('int')
df_vel.head()

In [ ]:
df_vel.tail(10)


In [ ]:
firstColName = t_sorted.columns[0]
#t_sorted = t_sorted.drop(columns = firstColName)
t_sorted.tail(10)


In [ ]:
len(t_sorted)

In [ ]:
len(df_vel)

In [ ]:
t_sorted_temp = t_sorted[['frame', 'particle', 'x']].apply(tuple, axis=1)
df_vel_temp = df_vel[['frame', 'particle', 'x']].apply(tuple, axis=1)

In [ ]:
extra_rows = t_sorted[~t_sorted_temp.isin(df_vel_temp)]

In [ ]:
extra_rows = extra_rows.sort_values(by = ['particle', 'frame'])
extra_rows.head(20)

In [ ]:
len(extra_rows)

In [ ]:
get_suku = t_large[(t_large['particle'] == 124) & (t_large['frame'] == 242)]
get_suku.head()

In [ ]:
get_suku_prev = t_large[(t_large['particle'] == 124) & (t_large['frame'] == (242 - step))]
get_suku_prev.head()

In [ ]:
valid_particles = np.unique(t_large[['particle', 'frame']].values - np.array([0, step]), axis=0)

In [ ]:
len(valid_particles)

In [ ]:
particle_time = t_sorted[['particle', 'frame']].values

In [ ]:
len(particle_time)

In [ ]:
mask = np.isin(particle_time, valid_particles).all(axis=1)

In [ ]:
len(mask)

In [ ]:
t_ultimate = t_sorted[mask]

In [ ]:
len(t_ultimate)

In [ ]:
get_suku = t_ultimate[(t_ultimate['particle'] == 171) & (t_ultimate['frame'] == 242)]
get_suku.head()

In [ ]:
get_suku_prev = t_large[(t_large['particle'] == 124) & (t_large['frame'] == (216))]
get_suku_prev.head()

In [ ]:
ind_particle = t_sorted[(t_sorted['particle'] == 124)]
ind_particle.head(10)

In [ ]:
np.max(ind_particle['frame'])

In [ ]:
ind_particle.tail(10)

In [ ]:
indiv_frames = np.array(ind_particle['frame'])
indiv_frames

In [ ]:
len(indiv_frames)